## Test different modeling approaches

In [1]:
import json

# Setup and imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
import xgboost as xgb
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import sys

_NOTEBOOK_DIR = Path().resolve()
_candidate_paths = [
    _NOTEBOOK_DIR,
    _NOTEBOOK_DIR / 'code',
    _NOTEBOOK_DIR.parent,
    _NOTEBOOK_DIR.parent / 'code',
]

for _candidate in _candidate_paths:
    if (_candidate / 'model_utils.py').exists():
        sys.path.insert(0, str(_candidate))
        break

from model_utils import (
    DEFAULT_EXCLUDE_COLS,
    prepare_features,
    evaluate_model,
    create_sample_weights,
    log_results_to_mlflow,
    calculate_top_k_overlap,
    compute_ranking_metrics,
)

# MLflow experiment setup
try:
    import mlflow
    MLFLOW_AVAILABLE = True
except ImportError:
    mlflow = None
    MLFLOW_AVAILABLE = False
    print("⚠️  MLflow not installed; skipping run tracking.")
else:
    MLFLOW_TRACKING_DIR = (_NOTEBOOK_DIR.parent / 'mlruns').resolve()
    mlflow.set_tracking_uri(f"file://{MLFLOW_TRACKING_DIR}")
    mlflow.set_experiment("box_office_modeling")

DEFAULT_MLFLOW_TAGS = {
    "project": "box_office",
    "notebook": "4_try_different_modeling_approaches",
}

def build_validation_snapshot(df: pd.DataFrame, target_col: str) -> pd.DataFrame:
    columns = []
    for col in ("title", "release_year"):
        if col in df.columns:
            columns.append(col)
    if target_col in df.columns and target_col not in columns:
        columns.append(target_col)
    if not columns:
        return df.copy()
    return df[columns].copy()

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [2]:
# Load the processed dataset
df = pd.read_csv('../data/dataset_domestic_processed.csv')

print(f"Dataset loaded: {df.shape[0]} movies, {df.shape[1]} features")
print(f"Date range: {df['release_year'].min()}-{df['release_year'].max()}")
print(f"Revenue range: ${df['revenue_domestic'].min():,.0f} - ${df['revenue_domestic'].max():,.0f}")
print(f"Average revenue: ${df['revenue_domestic'].mean():,.0f}")

# Display basic info
print(f"\nRevenue distribution by year:")
yearly_stats = df.groupby('release_year')['revenue_domestic'].agg(['count', 'mean', 'median', 'max']).round(0)
print(yearly_stats)

Dataset loaded: 2993 movies, 101 features
Date range: 2010-2026
Revenue range: $0 - $936,662,225
Average revenue: $44,469,300

Revenue distribution by year:
              count        mean      median          max
release_year                                            
2010            146  64280337.0  40601215.0  415004880.0
2011            165  54908928.0  35061555.0  381447587.0
2012            156  63759683.0  37267171.0  623357910.0
2013            154  63044646.0  32392138.0  424668047.0
2014            168  57349216.0  27230730.0  350159020.0
2015            166  62167649.0  25231858.0  936662225.0
2016            160  61243682.0  27712245.0  533539991.0
2017            157  65146231.0  25584504.0  620181382.0
2018            168  62381201.0  24298948.0  700426566.0
2019            156  67733330.0  24967827.0  858373000.0
2020             60  20545342.0   6383931.0  206305244.0
2021             92  47130847.0  15845234.0  814866759.0
2022            106  66848185.0  14759212.0  

In [3]:
# IMPORTANT: Exclude 2024, 2025, and 2026 data from training to prevent data leakage
# 2024 = test set (for model evaluation)
# 2025 = evaluation set (for additional validation)
# 2026 = prediction set (for forecasting)
original_count = len(df)
df_training = df[df['release_year'] <= 2023].copy()
df_2024 = df[df['release_year'] == 2024].copy()
df_2025 = df[df['release_year'] == 2025].copy()
df_2026 = df[df['release_year'] == 2026].copy()

print(f"🚨 DATA LEAKAGE PREVENTION:")
print(f"   Original dataset: {original_count:,} movies")
print(f"   Training data (2010-2023): {len(df_training):,} movies") 
print(f"   2024 test data: {len(df_2024):,} movies (EXCLUDED from training)")
print(f"   2025 evaluation data: {len(df_2025):,} movies (EXCLUDED from training)")
print(f"   2026 prediction data: {len(df_2026):,} movies (EXCLUDED from training)")
print(f"   ✅ Using training period data only for model development")

# Use training data only for the rest of the notebook
df = df_training

print(f"\n📊 Training dataset summary:")
print(f"   Date range: {df['release_year'].min()}-{df['release_year'].max()}")
print(f"   Revenue range: ${df['revenue_domestic'].min():,.0f} - ${df['revenue_domestic'].max():,.0f}") 
print(f"   Average revenue: ${df['revenue_domestic'].mean():,.0f}")
print(f"   Models will be tested on 2024, evaluated on 2025, and predict 2026")

🚨 DATA LEAKAGE PREVENTION:
   Original dataset: 2,993 movies
   Training data (2010-2023): 1,994 movies
   2024 test data: 169 movies (EXCLUDED from training)
   2025 evaluation data: 128 movies (EXCLUDED from training)
   2026 prediction data: 702 movies (EXCLUDED from training)
   ✅ Using training period data only for model development

📊 Training dataset summary:
   Date range: 2010-2023
   Revenue range: $514,870 - $936,662,225
   Average revenue: $60,154,439
   Models will be tested on 2024, evaluated on 2025, and predict 2026


In [4]:
# Dataset scope toggles (edit as needed)
scope_config_path = _NOTEBOOK_DIR.parent / 'data' / 'dataset_scope.json'
DEFAULT_SCOPE = {
    'filter_major_studios': False,
    'filter_english_only': False,
}
if scope_config_path.exists():
    try:
        DEFAULT_SCOPE.update(json.loads(scope_config_path.read_text()))
        print(f"Loaded dataset scope defaults from {scope_config_path.name}: {DEFAULT_SCOPE}")
    except json.JSONDecodeError:
        print('⚠️ Could not parse dataset_scope.json; using notebook defaults.')

FILTER_MAJOR_STUDIOS = DEFAULT_SCOPE['filter_major_studios']
FILTER_ENGLISH_ONLY = DEFAULT_SCOPE['filter_english_only']

# Uncomment to override defaults in this notebook run
# FILTER_MAJOR_STUDIOS = True
# FILTER_ENGLISH_ONLY = True

print('🎛️ Modeling dataset scope:')
print(f"   • FILTER_MAJOR_STUDIOS = {FILTER_MAJOR_STUDIOS}")
print(f"   • FILTER_ENGLISH_ONLY  = {FILTER_ENGLISH_ONLY}")


Loaded dataset scope defaults from dataset_scope.json: {'filter_major_studios': False, 'filter_english_only': False, 'studio_note': 'all studios', 'language_note': 'all languages', 'row_count': 1994}
🎛️ Modeling dataset scope:
   • FILTER_MAJOR_STUDIOS = False
   • FILTER_ENGLISH_ONLY  = False


In [5]:

def apply_dataset_scope(dataframe: pd.DataFrame, *, filter_major: bool = False, filter_english: bool = False, verbose: bool = True):
    df_scoped = dataframe.copy()
    notes = []

    if filter_major:
        if 'is_major_studio' in df_scoped.columns:
            before = len(df_scoped)
            df_scoped = df_scoped[df_scoped['is_major_studio'] == 1].copy()
            notes.append(f"major studios only ({len(df_scoped):,} rows, dropped {before - len(df_scoped):,})")
        else:
            if verbose:
                print('⚠️ FILTER_MAJOR_STUDIOS=True but "is_major_studio" column missing; skipping filter.')
            notes.append('major filter skipped (column missing)')
    else:
        notes.append('all studios')

    if filter_english:
        if 'is_english' in df_scoped.columns:
            before = len(df_scoped)
            df_scoped = df_scoped[df_scoped['is_english'] == 1].copy()
            notes.append(f"english-only ({len(df_scoped):,} rows, dropped {before - len(df_scoped):,})")
        elif 'original_language' in df_scoped.columns:
            before = len(df_scoped)
            df_scoped = df_scoped[df_scoped['original_language'].fillna('').str.lower() == 'en'].copy()
            notes.append(f"english-only ({len(df_scoped):,} rows, dropped {before - len(df_scoped):,})")
        else:
            if verbose:
                print('⚠️ FILTER_ENGLISH_ONLY=True but language columns missing; skipping filter.')
            notes.append('language filter skipped (column missing)')
    else:
        notes.append('all languages')

    scope = {
        'filter_major_studios': bool(filter_major),
        'filter_english_only': bool(filter_english),
        'studio_note': notes[0],
        'language_note': notes[1],
        'row_count': len(df_scoped),
    }
    if verbose:
        print('🎛️ Applied dataset scope:')
        print(f"   - Studio scope: {notes[0]}")
        print(f"   - Language scope: {notes[1]}")
        print(f"   - Rows retained: {len(df_scoped):,}")
    return df_scoped, scope


In [6]:

# Apply dataset scope and persist for downstream notebooks
df, DATA_SCOPE = apply_dataset_scope(
    df,
    filter_major=FILTER_MAJOR_STUDIOS,
    filter_english=FILTER_ENGLISH_ONLY,
    verbose=True,
)

DATA_SCOPE_PATH = _NOTEBOOK_DIR.parent / 'data' / 'dataset_scope.json'
DATA_SCOPE_PATH.write_text(json.dumps({
    'filter_major_studios': FILTER_MAJOR_STUDIOS,
    'filter_english_only': FILTER_ENGLISH_ONLY,
    'studio_note': DATA_SCOPE.get('studio_note'),
    'language_note': DATA_SCOPE.get('language_note'),
    'row_count': DATA_SCOPE.get('row_count'),
}, indent=2))
print(f"📄 Saved dataset scope details to {DATA_SCOPE_PATH}")

filtered_data_path = _NOTEBOOK_DIR.parent / 'data' / 'dataset_domestic_processed_modeling.csv'
df.to_csv(filtered_data_path, index=False)
print(f"💾 Wrote filtered modeling dataset to {filtered_data_path}")


🎛️ Applied dataset scope:
   - Studio scope: all studios
   - Language scope: all languages
   - Rows retained: 1,994
📄 Saved dataset scope details to /Users/jasmineplows/Documents/California/Projects/box_office/data/dataset_scope.json
💾 Wrote filtered modeling dataset to /Users/jasmineplows/Documents/California/Projects/box_office/data/dataset_domestic_processed_modeling.csv


In [7]:
df

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,production_countries,release_year,production_country_codes,is_origin_usa,is_origin_uk_ie,is_origin_canada,is_origin_us_uk_ca,genres,title_normalized,domestic_revenue,rank,distributor,revenue_domestic,revenue,genre_names,is_disney,is_warner_bros,is_universal,is_sony,is_paramount,is_fox,is_mgm,is_lionsgate,is_major_studio,is_english,genre_drama,genre_comedy,genre_action,genre_thriller,genre_adventure,genre_romance,genre_horror,genre_crime,genre_science_fiction,genre_fantasy,genre_family,genre_animation,genre_mystery,genre_history,genre_war,genre_music,genre_western,is_action_adventure,is_action_sci_fi,is_comedy_romance,is_family_animation,primary_genre,genre_count,release_month,release_month_name,release_season,is_summer_blockbuster,is_holiday_release,is_oscar_season,is_holiday_proximity,nearest_holiday,days_to_holiday,is_christmas_proximity,is_thanksgiving_proximity,is_independence_day_proximity,is_memorial_labor_day_proximity,has_nearby_major_release,nearby_major_releases_count,nearby_major_releases_max_revenue,days_to_nearest_major_release,competition_intensity,has_blockbuster_competition,has_immediate_competition,is_sequel_title,is_marvel,is_dc,is_star_wars,is_fast_furious,is_harry_potter,is_franchise_sequel,is_sequel,is_live_action_remake,is_adaptation,is_superhero,has_remake_indicator,is_remake_adaptation,is_ip_movie,years_since_baseline,is_pre_streaming_era,is_streaming_transition,is_pandemic_year,is_post_pandemic_era
0,False,/wE5JGzujfvDPMIfFjJyrhXFjZLc.jpg,"[16, 10751, 35]",10193,en,Toy Story 3,"Woody, Buzz, and the rest of Andy's toys haven...",14.5494,/AbbXspMOwdvwWZgVN0nabZq03Ec.jpg,2010-06-18,Toy Story 3,False,7.802,15182,"[{'iso_3166_1': 'US', 'name': 'United States o...",2010,['US'],1,0,0,1,"[16, 10751, 35]",Toy Story 3,415004880.0,42,Walt Disney Studios Motion Pictures,415004880.0,415004880.0,"Animation,Family,Comedy",1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,Animation,3,6,June,Summer,1,0,0,0,NaN,NaN,0,0,0,0,1,7,300531751,5.0,High,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-5,1,0,0,0
1,False,/ocOWbTzHcJVTw9Tz173KPeskDOP.jpg,"[10751, 14, 12]",12155,en,Alice in Wonderland,"Alice, now 19 years old, returns to the whimsi...",12.8459,/o0kre9wRCZz3jjSjaru7QU0UtFz.jpg,2010-03-05,Alice in Wonderland,False,6.636,14397,"[{'iso_3166_1': 'US', 'name': 'United States o...",2010,['US'],1,0,0,1,"[10751, 14, 12]",Alice in Wonderland,334191110.0,87,Walt Disney Studios Motion Pictures,334191110.0,334191110.0,"Family,Fantasy,Adventure",1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,Family,3,3,March,Spring,0,0,0,0,NaN,NaN,0,0,0,0,1,3,128012934,14.0,High,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,-5,1,0,0,0
2,False,/7lmBufEG7P7Y1HClYK3gCxYrkgS.jpg,"[12, 28, 878]",10138,en,Iron Man 2,With the world now aware of his dual life as t...,18.6595,/6WBeq4fCfn7AN0o21W9qNcRF2l9.jpg,2010-05-07,Iron Man 2,False,6.848,21737,"[{'iso_3166_1': 'US', 'name': 'United States o...",2010,['US'],1,0,0,1,"[12, 28, 878]",Iron Man 2,312433331.0,104,Paramount Pictures,312433331.0,312433331.0,"Adventure,Action,Science Fiction",0,0,0,0,1,0,0,0,1,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,Adventure,3,5,May,Spring,1,0,0,0,NaN,NaN,0,0,0,0,1,4,238736787,7.0,High,1,1,1,1,0,0,0,0,0,1,0,0,1,0,0,1,-5,1,0,0,0
3,False,/oguoQWtQJcqIkmmUNbe3CmLx6AP.jpg,"[12, 14, 18, 10749]",24021,en,The Twilight Saga: Eclipse,Bella once again finds herself surrounded by d...,16.3754,/dK4Gi1UdMiHzHc7r7CZQG4IQ9Sr.jpg,2010-06-23,The Twilight Saga: Eclipse,False,6.203,8992,"[{'iso_3166_1': 'US', 'name': 'United States o...",2010,['US'],1,0,0,1,"[12, 14, 18, 10749]",The Twilight Saga: Eclipse,300531751.0,112,Summit Entertainment,300531751.0,300531751.0,"Adventure,Fantasy,Drama,Romance",0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,Adventure,4,6,June,Summer,1,0,0,0,NaN,NaN,0,0,0,0,1,6,415004880,0.0,High,1,1,1,0,0,0,0,0,0

### Define Features to Use

In [8]:
# Prepare feature matrix using shared helper

df, feature_cols, target = prepare_features(
    df,
    exclude_cols=DEFAULT_EXCLUDE_COLS,
    filter_major_only=False,
    drop_major_flag_from_features=False,
    verbose=True,
)

print(f"Target variable: {target}")
print(f"Missing values in target: {df[target].isnull().sum()}")
print(f"Total features available: {len(feature_cols)}")



🔧 Feature preparation summary:
   • Candidate features: 69
   • Excluded non-numeric columns:
     – production_countries: sample values ["[{'iso_3166_1': 'US', 'name': 'United States of America'}]", "[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]", "[{'iso_3166_1': 'CN', 'name': 'China'}, {'iso_3166_1': 'HK', 'name': 'Hong Kong'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]", "[{'iso_3166_1': 'US', 'name': 'United States of America'}, {'iso_3166_1': 'GB', 'name': 'United Kingdom'}]", "[{'iso_3166_1': 'US', 'name': 'United States of America'}, {'iso_3166_1': 'BG', 'name': 'Bulgaria'}, {'iso_3166_1': 'DE', 'name': 'Germany'}]"]
     – production_country_codes: sample values ["['US']", "['GB', 'US']", "['HK', 'CN', 'US']", "['BG', 'DE', 'US']", "['NZ', 'US']"]
   • Columns with missing values:
     – days_to_holiday: 985 nulls
Target variable: revenue_domestic
Missing values in target: 0
Total features available: 

## 🔵 Approach 1: Ridge Regression (Linear Models)

**Why Ridge Regression?**
- Strong baseline for box office prediction
- Handles multicollinearity well (important with 63 features)
- Interpretable coefficients show feature importance
- Regularization prevents overfitting

**Experimental Design:**
- Test 3 pandemic handling strategies: All Years, No Pandemic, Weighted (30%)
- Hyperparameter tuning with Grid Search
- Compare strategies on 2024 validation data

**Expected Outcome**: Find best Ridge configuration and pandemic strategy for final comparison.

### Multiple Training Strategies

In [9]:
# Split data for validation
# Training: 2010-2023 (already filtered in df)
# Validation: 2024 (test set)
# Test: 2025 (evaluation set)

# FIXED: Use the separated dataframes we created earlier instead of filtered df
val_data = df_2024.copy()  # Validation data: use 2024 data (our test set)
test_data = df_2025.copy()  # Test data: use 2025 data (our evaluation set)

print(f"Training data shape: {df.shape}")
print(f"Validation data shape: {val_data.shape}")
print(f"Test data shape: {test_data.shape}")

# Prepare features and target for all sets
feature_columns = [col for col in df.columns if col not in ['revenue_domestic', 'title', 'release_date']]

X_train = df[feature_columns]
y_train = df['revenue_domestic']

X_val = val_data[feature_columns]
y_val = val_data['revenue_domestic']

X_test = test_data[feature_columns]
y_test = test_data['revenue_domestic']

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")

Training data shape: (1994, 101)
Validation data shape: (169, 101)
Test data shape: (128, 101)
X_train shape: (1994, 98)
X_val shape: (169, 98)
X_test shape: (128, 98)


In [10]:
print("""📊 Metrics Recap. 
- RMSE (Root Mean Squared Error): Average size of errors, penalizing large errors more heavily.  
- MAE (Mean Absolute Error): Average size of errors in plain dollar terms.  
- MAPE (Mean Absolute Percentage Error): Average percentage error. Over 100% means predictions are, on average, larger than the true values. 
- R² (Coefficient of Determination): Measures how much variance is explained by the model (0 = no explanatory power, 1 = perfect prediction).""")

# Train models with all three strategies using alternative time features
strategy_results = []

# Strategy 1: All eras including pandemic
results_all = train_strategy_models(train_data_all, "All Eras (2015-2023)")
strategy_results.append(results_all)

# Strategy 2: Exclude pandemic era
results_no_pandemic = train_strategy_models(train_data_no_pandemic, "No Pandemic Era")
strategy_results.append(results_no_pandemic)

# Strategy 3: Weighted approach (reduced pandemic influence)
results_weighted = train_strategy_models(train_data_weighted, "Weighted (30% pandemic era)", sample_weights)
strategy_results.append(results_weighted)

📊 Metrics Recap. 
- RMSE (Root Mean Squared Error): Average size of errors, penalizing large errors more heavily.  
- MAE (Mean Absolute Error): Average size of errors in plain dollar terms.  
- MAPE (Mean Absolute Percentage Error): Average percentage error. Over 100% means predictions are, on average, larger than the true values. 
- R² (Coefficient of Determination): Measures how much variance is explained by the model (0 = no explanatory power, 1 = perfect prediction).


NameError: name 'train_strategy_models' is not defined

In [ ]:
# Compare strategies
print("\n📊 Strategy Comparison Results")
print("=" * 70)

strategy_comparison = pd.DataFrame([
    {
        'Strategy': r['strategy'],
        'RMSE': f"${r['rmse']:,.0f}",
        'MAE': f"${r['mae']:,.0f}",
        'MAPE': f"{r['mape']:.1f}%",
        'R²': f"{r['r2']:.3f}"
    }
    for r in strategy_results
])

print(strategy_comparison.to_string(index=False))

# Find best strategy
best_strategy_idx = min(range(len(strategy_results)), key=lambda i: strategy_results[i]['rmse'])
best_strategy = strategy_results[best_strategy_idx]

print(f"\n🏆 Best Strategy: {best_strategy['strategy']}")
print(f"   Validation RMSE: ${best_strategy['rmse']:,.0f}")
print(f"   This will be used for 2026 predictions")

# Store best model
best_model_pandemic_aware = best_strategy['model_object']

# Visualize strategy comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

metrics = ['rmse', 'mape', 'r2']
metric_names = ['RMSE ($M)', 'MAPE (%)', 'R² Score']
colors = ['lightcoral', 'skyblue', 'lightgreen']

for i, (metric, name, color) in enumerate(zip(metrics, metric_names, colors)):
    values = [r[metric] for r in strategy_results]
    strategies = [r['strategy'] for r in strategy_results]
    
    if metric == 'rmse':
        values = [v/1e6 for v in values]  # Convert to millions for readability
    
    bars = axes[i].bar(strategies, values, color=color, alpha=0.7)
    axes[i].set_title(f'{name} by Strategy')
    axes[i].tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, val in zip(bars, values):
        height = bar.get_height()
        axes[i].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                    f'{val:.1f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("✅ Pandemic-aware model selection complete")

## 🌳 Approach 2: Decision Trees (Non-Linear Models)

**Why Decision Trees?**
- Captures non-linear relationships and feature interactions
- Naturally handles mixed data types and feature importance
- No assumption about data distribution
- Interpretable decision rules

**Experimental Design:**
- Same 3 pandemic strategies as Ridge (All Years, No Pandemic, Weighted)
- Hyperparameter tuning: max_depth, min_samples_split, min_samples_leaf, max_features
- Feature importance analysis to understand key predictors
- Compare against Ridge baseline

**Expected Outcome**: Test whether non-linear modeling improves over Ridge regression.

In [ ]:

# Import Decision Tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree

print("🌳 Setting up Decision Tree Regression Models")

def train_decision_tree_strategies(train_data, strategy_name, sample_weights=None):
    """Train Decision Tree models with a specific strategy"""
    print(f"🌳 Training Decision Tree - {strategy_name}")

    X_train_strat = train_data[feature_cols]
    y_train_strat = train_data[target]
    y_train_strat_log = np.log1p(y_train_strat)

    X_val = val_data[feature_cols]
    y_val = val_data[target]
    y_val_log = np.log1p(y_val)

    # Decision Tree pipeline (no scaling needed for trees)
    dt_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('dt', DecisionTreeRegressor(random_state=42))
    ])

    # Hyperparameter grid for Decision Tree
    dt_params = {
        'dt__max_depth': [5, 6, 7, 8, 9, 10],
        'dt__min_samples_split': [10, 20, 50],
        'dt__min_samples_leaf': [5, 10, 15],
        'dt__max_features': ['sqrt', 'log2', None]
    }

    dt_grid = GridSearchCV(
        dt_pipeline, dt_params, cv=5, 
        scoring='neg_mean_squared_error', n_jobs=-1
    )

    # Fit with sample weights if provided
    if sample_weights is not None:
        # GridSearchCV doesn't directly support sample weights for Decision Trees
        # We'll use best params from unweighted search, then refit with weights
        dt_grid.fit(X_train_strat, y_train_strat_log)
        best_params = dt_grid.best_params_

        # Refit with weights
        dt_weighted = Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('dt', DecisionTreeRegressor(
                max_depth=best_params['dt__max_depth'],
                min_samples_split=best_params['dt__min_samples_split'],
                min_samples_leaf=best_params['dt__min_samples_leaf'],
                max_features=best_params['dt__max_features'],
                random_state=42
            ))
        ])
        dt_weighted.fit(X_train_strat, y_train_strat_log, dt__sample_weight=sample_weights)
        dt_best = dt_weighted
        print(f"  Best params: {best_params}")
    else:
        dt_grid.fit(X_train_strat, y_train_strat_log)
        dt_best = dt_grid.best_estimator_
        best_params = dt_grid.best_params_
        print(f"  Best params: {best_params}")

    # Evaluate
    results = evaluate_model(dt_best, X_val, y_val_log, y_val, f"Decision Tree - {strategy_name}")
    results['strategy'] = strategy_name
    results['model_object'] = dt_best
    results['model_type'] = 'Decision Tree'

    major_only_flag = None
    if 'is_major_studio' in train_data.columns:
        major_only_flag = bool(train_data['is_major_studio'].fillna(0).eq(1).all())

    english_only_flag = None
    if 'original_language' in train_data.columns:
        english_only_flag = bool(train_data['original_language'].fillna('').eq('en').all())

        extra_metrics = {}
    ranking_metrics = compute_ranking_metrics(
        val_data,
        results.get('predictions'),
        target_col=target,
        title_col='title',
        k=10,
    )
    if ranking_metrics:
        if 'top10_overlap' in ranking_metrics:
            results['top10_overlap'] = int(ranking_metrics['top10_overlap'])
        for key, value in ranking_metrics.items():
            if key == 'top10_overlap':
                continue
            results[key] = value
        for key, value in ranking_metrics.items():
            if value is not None:
                extra_metrics[key] = float(value)

    if MLFLOW_AVAILABLE:
            tuned_params = {k.replace('dt__', ''): v for k, v in best_params.items()}
            params_to_log = {
                'model': 'DecisionTreeRegressor',
                'uses_sample_weights': sample_weights is not None,
                'n_features': len(feature_cols),
                'train_rows': len(train_data),
                'val_rows': len(val_data),
                **tuned_params,
            }
            tags = dict(DEFAULT_MLFLOW_TAGS, strategy=strategy_name, model_family='decision_tree')
            validation_snapshot = build_validation_snapshot(val_data, target)
            log_results_to_mlflow(
                results,
                run_name=f"DecisionTree | {strategy_name}",
                params=params_to_log,
                tags=tags,
                extra_metrics=extra_metrics or None,
                feature_frame=X_val,
                validation_frame=validation_snapshot,
                dataset_major_only=major_only_flag,
                dataset_english_only=english_only_flag,
            )

    return results

print("✅ Decision Tree framework ready")


In [ ]:
# Train Decision Tree models with all three strategies
dt_strategy_results = []

# Strategy 1: All eras including pandemic
dt_results_all = train_decision_tree_strategies(train_data_all, "All Eras (2015-2023)")
dt_strategy_results.append(dt_results_all)

# Strategy 2: Exclude pandemic era
dt_results_no_pandemic = train_decision_tree_strategies(train_data_no_pandemic, "No Pandemic Era")
dt_strategy_results.append(dt_results_no_pandemic)

# Strategy 3: Weighted approach (reduced pandemic influence)
dt_results_weighted = train_decision_tree_strategies(train_data_weighted, "Weighted (30% pandemic era)", sample_weights)
dt_strategy_results.append(dt_results_weighted)

In [ ]:
# Compare Decision Tree strategies
print("\n📊 Decision Tree Strategy Comparison")
print("=" * 70)

dt_strategy_comparison = pd.DataFrame([
    {
        'Strategy': r['strategy'],
        'RMSE': f"${r['rmse']:,.0f}",
        'MAE': f"${r['mae']:,.0f}",
        'MAPE': f"{r['mape']:.1f}%",
        'R²': f"{r['r2']:.3f}"
    }
    for r in dt_strategy_results
])

print(dt_strategy_comparison.to_string(index=False))

# Find best Decision Tree strategy
best_dt_strategy_idx = min(range(len(dt_strategy_results)), key=lambda i: dt_strategy_results[i]['rmse'])
best_dt_strategy = dt_strategy_results[best_dt_strategy_idx]

print(f"\n🏆 Best Decision Tree Strategy: {best_dt_strategy['strategy']}")
print(f"   Validation RMSE: ${best_dt_strategy['rmse']:,.0f}")

# Store best Decision Tree model
best_dt_model = best_dt_strategy['model_object']

In [ ]:
# Feature Importance Analysis (unique to Decision Trees)
print("\n🔍 Feature Importance Analysis")
print("=" * 50)

# Get feature importances from best Decision Tree model
dt_model = best_dt_model.named_steps['dt']
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': dt_model.feature_importances_
}).sort_values('importance', ascending=False)

# Top 15 most important features
top_features = feature_importance.head(15)
print("Top 15 Most Important Features:")
print(top_features.to_string(index=False))

# Visualize feature importance
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top 15 features
axes[0].barh(top_features['feature'][::-1], top_features['importance'][::-1], color='forestgreen', alpha=0.7)
axes[0].set_title('Top 15 Feature Importances (Decision Tree)')
axes[0].set_xlabel('Importance')

# Feature importance distribution
axes[1].hist(feature_importance['importance'], bins=20, color='forestgreen', alpha=0.7, edgecolor='black')
axes[1].set_title('Distribution of Feature Importances')
axes[1].set_xlabel('Importance')
axes[1].set_ylabel('Number of Features')

plt.tight_layout()
plt.show()

print(f"\n📈 Feature Analysis Summary:")
print(f"   Total features: {len(feature_cols)}")
print(f"   Features with importance > 0.01: {(feature_importance['importance'] > 0.01).sum()}")
print(f"   Top 5 features account for: {top_features['importance'].head(5).sum():.1%} of total importance")

## 🌲 Approach 3: Random Forest (Ensemble Models)

**Why Random Forest?**
- Ensemble of decision trees reduces overfitting
- Better generalization than single decision trees
- Built-in feature importance and variance reduction
- Robust to outliers and noise

**Experimental Design:**
- Same 3 pandemic strategies as previous approaches
- Extensive hyperparameter tuning: n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features
- Feature importance analysis and comparison with Decision Tree
- Bootstrap aggregation for improved stability

**Expected Outcome**: Test whether ensemble methods provide the best performance for box office prediction.

In [ ]:

# Import Random Forest
from sklearn.ensemble import RandomForestRegressor

print("🌲 Setting up Random Forest Regression Models")

def train_random_forest_strategies(train_data, strategy_name, sample_weights=None):
    """Train Random Forest models with a specific strategy"""
    print(f"🌲 Training Random Forest - {strategy_name}")

    X_train_strat = train_data[feature_cols]
    y_train_strat = train_data[target]
    y_train_strat_log = np.log1p(y_train_strat)

    X_val = val_data[feature_cols]
    y_val = val_data[target]
    y_val_log = np.log1p(y_val)

    # Random Forest pipeline (no scaling needed for tree-based models)
    rf_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('rf', RandomForestRegressor(random_state=42, n_jobs=-1))
    ])

    # Hyperparameter grid for Random Forest
    rf_params = {
        'rf__n_estimators': [100, 150, 200],  # Start with 100 as requested
        'rf__max_depth': [6, 8, 10, 12, None],
        'rf__min_samples_split': [10, 20, 50],
        'rf__min_samples_leaf': [5, 10, 15],
        'rf__max_features': ['sqrt', 'log2', 0.3]
    }

    rf_grid = GridSearchCV(
        rf_pipeline, rf_params, cv=5, 
        scoring='neg_mean_squared_error', n_jobs=-1, verbose=1
    )

    # Fit with sample weights if provided
    if sample_weights is not None:
        # GridSearchCV doesn't directly support sample weights for Random Forest
        # We'll use best params from unweighted search, then refit with weights
        print("  Running hyperparameter search...")
        rf_grid.fit(X_train_strat, y_train_strat_log)
        best_params = rf_grid.best_params_

        # Refit with weights
        print("  Refitting with sample weights...")
        rf_weighted = Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('rf', RandomForestRegressor(
                n_estimators=best_params['rf__n_estimators'],
                max_depth=best_params['rf__max_depth'],
                min_samples_split=best_params['rf__min_samples_split'],
                min_samples_leaf=best_params['rf__min_samples_leaf'],
                max_features=best_params['rf__max_features'],
                random_state=42,
                n_jobs=-1
            ))
        ])
        rf_weighted.fit(X_train_strat, y_train_strat_log, rf__sample_weight=sample_weights)
        rf_best = rf_weighted
        print(f"  Best params: {best_params}")
    else:
        print("  Running hyperparameter search...")
        rf_grid.fit(X_train_strat, y_train_strat_log)
        rf_best = rf_grid.best_estimator_
        best_params = rf_grid.best_params_
        print(f"  Best params: {best_params}")

    # Evaluate
    results = evaluate_model(rf_best, X_val, y_val_log, y_val, f"Random Forest - {strategy_name}")
    results['strategy'] = strategy_name
    results['model_object'] = rf_best
    results['model_type'] = 'Random Forest'

    major_only_flag = None
    if 'is_major_studio' in train_data.columns:
        major_only_flag = bool(train_data['is_major_studio'].fillna(0).eq(1).all())

    english_only_flag = None
    if 'original_language' in train_data.columns:
        english_only_flag = bool(train_data['original_language'].fillna('').eq('en').all())

        extra_metrics = {}
    ranking_metrics = compute_ranking_metrics(
        val_data,
        results.get('predictions'),
        target_col=target,
        title_col='title',
        k=10,
    )
    if ranking_metrics:
        if 'top10_overlap' in ranking_metrics:
            results['top10_overlap'] = int(ranking_metrics['top10_overlap'])
        for key, value in ranking_metrics.items():
            if key == 'top10_overlap':
                continue
            results[key] = value
        for key, value in ranking_metrics.items():
            if value is not None:
                extra_metrics[key] = float(value)

    if MLFLOW_AVAILABLE:
            tuned_params = {k.replace('rf__', ''): v for k, v in best_params.items()}
            params_to_log = {
                'model': 'RandomForestRegressor',
                'uses_sample_weights': sample_weights is not None,
                'n_features': len(feature_cols),
                'train_rows': len(train_data),
                'val_rows': len(val_data),
                **tuned_params,
            }
            tags = dict(DEFAULT_MLFLOW_TAGS, strategy=strategy_name, model_family='random_forest')
            validation_snapshot = build_validation_snapshot(val_data, target)
            log_results_to_mlflow(
                results,
                run_name=f"RandomForest | {strategy_name}",
                params=params_to_log,
                tags=tags,
                extra_metrics=extra_metrics or None,
                feature_frame=X_val,
                validation_frame=validation_snapshot,
                dataset_major_only=major_only_flag,
                dataset_english_only=english_only_flag,
            )

    return results

print("✅ Random Forest framework ready")


In [ ]:
# Train Random Forest models with all three strategies 
print("🎯 Training Random Forest Models")

rf_strategy_results = []

# Strategy 1: All eras including pandemic
rf_results_all = train_random_forest_strategies(train_data_all, "All Eras (2015-2023)")
rf_strategy_results.append(rf_results_all)

# Strategy 2: Exclude pandemic era
rf_results_no_pandemic = train_random_forest_strategies(train_data_no_pandemic, "No Pandemic Era")
rf_strategy_results.append(rf_results_no_pandemic)

# Strategy 3: Weighted approach (reduced pandemic influence)
rf_results_weighted = train_random_forest_strategies(train_data_weighted, "Weighted (30% pandemic era)", sample_weights)
rf_strategy_results.append(rf_results_weighted)

In [ ]:
# Compare Random Forest strategies
print("\n📊 Random Forest Strategy Comparison")
print("=" * 70)

rf_strategy_comparison = pd.DataFrame([
    {
        'Strategy': r['strategy'],
        'RMSE': f"${r['rmse']:,.0f}",
        'MAE': f"${r['mae']:,.0f}",
        'MAPE': f"{r['mape']:.1f}%",
        'R²': f"{r['r2']:.3f}"
    }
    for r in rf_strategy_results
])

print(rf_strategy_comparison.to_string(index=False))

# Find best Random Forest strategy
best_rf_strategy_idx = min(range(len(rf_strategy_results)), key=lambda i: rf_strategy_results[i]['rmse'])
best_rf_strategy = rf_strategy_results[best_rf_strategy_idx]

print(f"\n🏆 Best Random Forest Strategy: {best_rf_strategy['strategy']}")
print(f"   Validation RMSE: ${best_rf_strategy['rmse']:,.0f}")

# Store best Random Forest model
best_rf_model = best_rf_strategy['model_object']

In [ ]:
# Random Forest Feature Importance Analysis
print("\n🔍 Random Forest Feature Importance Analysis")
print("=" * 60)

# Get feature importances from best Random Forest model
rf_model = best_rf_model.named_steps['rf']
rf_feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# Top 15 most important features
rf_top_features = rf_feature_importance.head(15)
print("Top 15 Most Important Features (Random Forest):")
print(rf_top_features.to_string(index=False))

# Compare with Decision Tree feature importance
dt_model = best_dt_model.named_steps['dt']
dt_feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': dt_model.feature_importances_
}).sort_values('importance', ascending=False)

# Visualize feature importance comparison
fig, axes = plt.subplots(2, 2, figsize=(20, 12))
fig.suptitle('Feature Importance Analysis: Random Forest vs Decision Tree', fontsize=16, fontweight='bold')

# Top 15 Random Forest features
axes[0,0].barh(rf_top_features['feature'][::-1], rf_top_features['importance'][::-1], 
               color='darkgreen', alpha=0.7)
axes[0,0].set_title('Top 15 Feature Importances (Random Forest)')
axes[0,0].set_xlabel('Importance')

# Top 15 Decision Tree features (for comparison)
dt_top_features = dt_feature_importance.head(15)
axes[0,1].barh(dt_top_features['feature'][::-1], dt_top_features['importance'][::-1], 
               color='forestgreen', alpha=0.7)
axes[0,1].set_title('Top 15 Feature Importances (Decision Tree)')
axes[0,1].set_xlabel('Importance')

# Feature importance distribution comparison
axes[1,0].hist(rf_feature_importance['importance'], bins=20, alpha=0.7, 
               color='darkgreen', label='Random Forest', edgecolor='black')
axes[1,0].hist(dt_feature_importance['importance'], bins=20, alpha=0.5, 
               color='orange', label='Decision Tree', edgecolor='black')
axes[1,0].set_title('Feature Importance Distribution Comparison')
axes[1,0].set_xlabel('Importance')
axes[1,0].set_ylabel('Number of Features')
axes[1,0].legend()

# Feature importance correlation between models
# Create merged dataframe for comparison
importance_comparison = pd.merge(
    rf_feature_importance[['feature', 'importance']].rename(columns={'importance': 'rf_importance'}),
    dt_feature_importance[['feature', 'importance']].rename(columns={'importance': 'dt_importance'}),
    on='feature'
)

axes[1,1].scatter(importance_comparison['dt_importance'], importance_comparison['rf_importance'], 
                 alpha=0.6, s=50)
axes[1,1].set_xlabel('Decision Tree Importance')
axes[1,1].set_ylabel('Random Forest Importance')
axes[1,1].set_title('Feature Importance Correlation\n(Random Forest vs Decision Tree)')

# Add diagonal line
max_importance = max(importance_comparison['dt_importance'].max(), 
                    importance_comparison['rf_importance'].max())
axes[1,1].plot([0, max_importance], [0, max_importance], 'r--', alpha=0.5)

# Calculate correlation
correlation = importance_comparison['dt_importance'].corr(importance_comparison['rf_importance'])
axes[1,1].text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
               transform=axes[1,1].transAxes, bbox=dict(boxstyle="round,pad=0.3", facecolor="white"))

plt.tight_layout()
plt.show()

print(f"\n📈 Random Forest Feature Analysis Summary:")
print(f"   Total features: {len(feature_cols)}")
print(f"   Features with importance > 0.01: {(rf_feature_importance['importance'] > 0.01).sum()}")
print(f"   Top 5 features account for: {rf_top_features['importance'].head(5).sum():.1%} of total importance")
print(f"   RF vs DT feature importance correlation: {correlation:.3f}")

# Show top features that differ between models
print(f"\n🔄 Feature Ranking Differences (RF vs DT):")
rf_ranks = {feat: i for i, feat in enumerate(rf_feature_importance['feature'])}
dt_ranks = {feat: i for i, feat in enumerate(dt_feature_importance['feature'])}

rank_differences = []
for feat in feature_cols:
    rf_rank = rf_ranks[feat]
    dt_rank = dt_ranks[feat]
    rank_diff = abs(rf_rank - dt_rank)
    rank_differences.append({'feature': feat, 'rf_rank': rf_rank, 'dt_rank': dt_rank, 'rank_diff': rank_diff})

rank_differences = pd.DataFrame(rank_differences).sort_values('rank_diff', ascending=False)
print("Top 10 features with largest ranking differences:")
print(rank_differences.head(10)[['feature', 'rf_rank', 'dt_rank', 'rank_diff']].to_string(index=False))

## 🚀 Approach 4: XGBoost (Gradient Boosting)

**Why XGBoost?**
- State-of-the-art gradient boosting framework
- Excellent performance on structured/tabular data
- Built-in regularization and feature importance
- Handles missing values automatically
- Often wins machine learning competitions

**Experimental Design:**
- Same 3 pandemic strategies as previous approaches
- Extensive hyperparameter tuning: n_estimators, max_depth, learning_rate, subsample, colsample_bytree
- Feature importance analysis and comparison with tree-based models
- Built-in cross-validation for early stopping

**Expected Outcome**: Test whether gradient boosting provides superior performance for box office prediction.

In [ ]:
print("🚀 Setting up XGBoost Regression Models")

def train_xgboost_strategies(train_data, strategy_name, sample_weights=None):
    """Train XGBoost models with a specific strategy"""
    print(f"🚀 Training XGBoost - {strategy_name}")

    X_train_strat = train_data[feature_cols]
    y_train_strat = train_data[target]
    y_train_strat_log = np.log1p(y_train_strat)

    X_val = val_data[feature_cols]
    y_val = val_data[target]
    y_val_log = np.log1p(y_val)

    # XGBoost pipeline
    xgb_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('xgb', xgb.XGBRegressor(random_state=42, verbosity=0))
    ])

    # Hyperparameter grid
    xgb_params = {
        'xgb__n_estimators': [100, 200, 300],
        'xgb__max_depth': [3, 4, 5, 6],
        'xgb__learning_rate': [0.01, 0.05, 0.1, 0.2],
        'xgb__subsample': [0.8, 0.9, 1.0],
        'xgb__colsample_bytree': [0.8, 0.9, 1.0]
    }

    xgb_grid = GridSearchCV(
        xgb_pipeline, xgb_params, cv=5, 
        scoring='neg_mean_squared_error', n_jobs=-1, verbose=1
    )

    # Fit with or without sample weights
    if sample_weights is not None:
        print("  Running hyperparameter search...")
        xgb_grid.fit(X_train_strat, y_train_strat_log)
        best_params = xgb_grid.best_params_

        print("  Refitting with sample weights...")
        xgb_weighted = Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('xgb', xgb.XGBRegressor(
                n_estimators=best_params['xgb__n_estimators'],
                max_depth=best_params['xgb__max_depth'],
                learning_rate=best_params['xgb__learning_rate'],
                subsample=best_params['xgb__subsample'],
                colsample_bytree=best_params['xgb__colsample_bytree'],
                random_state=42,
                verbosity=0
            ))
        ])
        xgb_weighted.fit(X_train_strat, y_train_strat_log, xgb__sample_weight=sample_weights)
        xgb_best = xgb_weighted
        print(f"  Best params: {best_params}")
    else:
        print("  Running hyperparameter search...")
        xgb_grid.fit(X_train_strat, y_train_strat_log)
        xgb_best = xgb_grid.best_estimator_
        best_params = xgb_grid.best_params_
        print(f"  Best params: {best_params}")

    # Evaluate
    results = evaluate_model(xgb_best, X_val, y_val_log, y_val, f"XGBoost - {strategy_name}")
    results['strategy'] = strategy_name
    results['model_object'] = xgb_best
    results['model_type'] = 'XGBoost'

    major_only_flag = None
    if 'is_major_studio' in train_data.columns:
        major_only_flag = bool(train_data['is_major_studio'].fillna(0).eq(1).all())

    english_only_flag = None
    if 'original_language' in train_data.columns:
        english_only_flag = bool(train_data['original_language'].fillna('').eq('en').all())

    extra_metrics = {}
    ranking_metrics = compute_ranking_metrics(
        val_data,
        results.get('predictions'),
        target_col=target,
        title_col='title',
        k=10,
    )
    if ranking_metrics:
        if 'top10_overlap' in ranking_metrics:
            results['top10_overlap'] = int(ranking_metrics['top10_overlap'])
        for key, value in ranking_metrics.items():
            if key == 'top10_overlap':
                continue
            results[key] = value
        for key, value in ranking_metrics.items():
            if value is not None:
                extra_metrics[key] = float(value)

    if MLFLOW_AVAILABLE:
        tuned_params = {k.replace('xgb__', ''): v for k, v in best_params.items()}
        params_to_log = {
            'model': 'XGBRegressor',
            'uses_sample_weights': sample_weights is not None,
            'n_features': len(feature_cols),
            'train_rows': len(train_data),
            'val_rows': len(val_data),
            **tuned_params,
        }
        tags = dict(DEFAULT_MLFLOW_TAGS, strategy=strategy_name, model_family='xgboost')
        validation_snapshot = build_validation_snapshot(val_data, target)
        log_results_to_mlflow(
            results,
            run_name=f"XGBoost | {strategy_name}",
            params=params_to_log,
            tags=tags,
            extra_metrics=extra_metrics or None,
            feature_frame=X_val,
            validation_frame=validation_snapshot,
            dataset_major_only=major_only_flag,
            dataset_english_only=english_only_flag,
        )

    return results  # ✅ correctly indented inside the function

print("✅ XGBoost framework ready")

In [ ]:

# Train XGBoost models with all three strategies 
print("🎯 Training XGBoost Models")

xgb_strategy_results = []

# Strategy 1: All eras including pandemic
xgb_results_all = train_xgboost_strategies(train_data_all, "All Eras (2015-2023)")
xgb_strategy_results.append(xgb_results_all)

# Strategy 2: Exclude pandemic era
xgb_results_no_pandemic = train_xgboost_strategies(train_data_no_pandemic, "No Pandemic Era")
xgb_strategy_results.append(xgb_results_no_pandemic)

# Strategy 3: Weighted approach (reduced pandemic influence)
xgb_results_weighted = train_xgboost_strategies(train_data_weighted, "Weighted (30% pandemic era)", sample_weights)
xgb_strategy_results.append(xgb_results_weighted)

# Compare XGBoost strategies
print("\n📊 XGBoost Strategy Comparison")
print("=" * 70)

xgb_strategy_comparison = pd.DataFrame([
    {
        'Strategy': r['strategy'],
        'RMSE': f"${r['rmse']:,.0f}",
        'MAE': f"${r['mae']:,.0f}",
        'MAPE': f"{r['mape']:.1f}%",
        'R²': f"{r['r2']:.3f}"
    }
    for r in xgb_strategy_results
])

print(xgb_strategy_comparison.to_string(index=False))

# Find best XGBoost strategy
best_xgb_strategy_idx = min(range(len(xgb_strategy_results)), key=lambda i: xgb_strategy_results[i]['rmse'])
best_xgb_strategy = xgb_strategy_results[best_xgb_strategy_idx]

print(f"\n🏆 Best XGBoost Strategy: {best_xgb_strategy['strategy']}")
print(f"   Validation RMSE: ${best_xgb_strategy['rmse']:,.0f}")

# Store best XGBoost model
best_xgb_model = best_xgb_strategy['model_object']

xgb_strategy_results = []

## 💡 Approach 5: LightGBM (Fast Gradient Boosting)

**Why LightGBM?**
- Microsoft's high-performance gradient boosting framework
- Faster training than XGBoost with similar accuracy
- Built-in categorical feature handling
- Memory efficient and scalable
- Often comparable or superior to XGBoost

**Experimental Design:**
- Same 3 pandemic strategies as previous approaches
- Hyperparameter tuning: n_estimators, max_depth, learning_rate, num_leaves, feature_fraction
- Feature importance analysis and comparison with other tree-based models
- Early stopping to prevent overfitting

**Expected Outcome**: Compare LightGBM performance against XGBoost and other models.

In [ ]:
# ============================================
# LightGBM: RandomizedSearchCV + Early Stopping
# ============================================

print("💡 Setting up LightGBM Regression Models (RandomizedSearchCV)")

from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

LGB_N_ITER = 40
LGB_CV_FOLDS = 5
LGB_ES_ROUNDS = 100
LGB_RANDOM_STATE = 42

def train_lightgbm_strategies(train_data, strategy_name, sample_weights=None):
    """Train LightGBM models with a specific strategy (RandomizedSearchCV)"""
    print(f"💡 Training LightGBM - {strategy_name}")

    # Split
    X_train_strat = train_data[feature_cols]
    y_train_strat = train_data[target]
    y_train_strat_log = np.log1p(y_train_strat)

    X_val = val_data[feature_cols]
    y_val = val_data[target]
    y_val_log = np.log1p(y_val)

    # Pipeline
    lgb_pipeline = Pipeline([
        ('lgb', lgb.LGBMRegressor(
            random_state=LGB_RANDOM_STATE,
            verbosity=-1,
            n_jobs=1
        ))
    ])

    # Search space
    lgb_param_dist = {
        'lgb__n_estimators': [500, 1000, 1500, 2000],
        'lgb__learning_rate': [0.02, 0.03, 0.05, 0.08],
        'lgb__num_leaves': [63, 95, 127, 191],
        'lgb__min_child_samples': [20, 50, 100, 200],
        'lgb__max_depth': [-1],
        'lgb__feature_fraction': [0.7, 0.8, 0.9, 1.0],
        'lgb__bagging_fraction': [0.7, 0.8, 0.9, 1.0],
        'lgb__bagging_freq': [1],
        'lgb__reg_lambda': [0.0, 1.0, 5.0, 10.0],
        'lgb__reg_alpha': [0.0, 0.1, 0.5, 1.0],
        'lgb__min_split_gain': [0.0, 0.05, 0.1]
    }

    lgb_rand = RandomizedSearchCV(
        estimator=lgb_pipeline,
        param_distributions=lgb_param_dist,
        n_iter=LGB_N_ITER,
        cv=LGB_CV_FOLDS,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=1,
        random_state=LGB_RANDOM_STATE
    )

    # Early stopping
    es_cb = lgb.early_stopping(stopping_rounds=LGB_ES_ROUNDS, verbose=False)
    fit_kwargs = {
        'lgb__eval_set': [(X_val, y_val_log)],
        'lgb__eval_metric': 'rmse',
        'lgb__callbacks': [es_cb]
    }

    print("  Running randomized search...")
    if sample_weights is None:
        lgb_rand.fit(X_train_strat, y_train_strat_log, **fit_kwargs)
        lgb_best = lgb_rand.best_estimator_
        best_params = lgb_rand.best_params_
        print(f"  Best params: {best_params}")
    else:
        lgb_rand.fit(X_train_strat, y_train_strat_log, **fit_kwargs)
        best_params = lgb_rand.best_params_
        print(f"  Best params: {best_params}")

        print("  Refitting with sample weights...")
        lgb_weighted = Pipeline([
            ('lgb', lgb.LGBMRegressor(
                n_estimators=best_params['lgb__n_estimators'],
                learning_rate=best_params['lgb__learning_rate'],
                num_leaves=best_params['lgb__num_leaves'],
                min_child_samples=best_params['lgb__min_child_samples'],
                max_depth=best_params['lgb__max_depth'],
                feature_fraction=best_params['lgb__feature_fraction'],
                bagging_fraction=best_params['lgb__bagging_fraction'],
                bagging_freq=best_params['lgb__bagging_freq'],
                reg_lambda=best_params['lgb__reg_lambda'],
                reg_alpha=best_params['lgb__reg_alpha'],
                min_split_gain=best_params['lgb__min_split_gain'],
                random_state=LGB_RANDOM_STATE,
                verbosity=-1,
                n_jobs=1
            ))
        ])
        lgb_weighted.fit(
            X_train_strat, y_train_strat_log,
            lgb__sample_weight=sample_weights,
            **fit_kwargs
        )
        lgb_best = lgb_weighted

    # Evaluate
    results = evaluate_model(lgb_best, X_val, y_val_log, y_val, f"LightGBM - {strategy_name}")
    results['strategy'] = strategy_name
    results['model_object'] = lgb_best
    results['model_type'] = 'LightGBM'

    major_only_flag = None
    if 'is_major_studio' in train_data.columns:
        major_only_flag = bool(train_data['is_major_studio'].fillna(0).eq(1).all())

    english_only_flag = None
    if 'original_language' in train_data.columns:
        english_only_flag = bool(train_data['original_language'].fillna('').eq('en').all())

    extra_metrics = {}
    ranking_metrics = compute_ranking_metrics(
        val_data,
        results.get('predictions'),
        target_col=target,
        title_col='title',
        k=10,
    )
    if ranking_metrics:
        if 'top10_overlap' in ranking_metrics:
            results['top10_overlap'] = int(ranking_metrics['top10_overlap'])
        for key, value in ranking_metrics.items():
            if key == 'top10_overlap':
                continue
            results[key] = value
        for key, value in ranking_metrics.items():
            if value is not None:
                extra_metrics[key] = float(value)

    if MLFLOW_AVAILABLE:
        tuned_params = {k.replace('lgb__', ''): v for k, v in best_params.items()}
        params_to_log = {
            'model': 'LGBMRegressor',
            'uses_sample_weights': sample_weights is not None,
            'n_features': len(feature_cols),
            'train_rows': len(train_data),
            'val_rows': len(val_data),
            **tuned_params,
        }
        tags = dict(DEFAULT_MLFLOW_TAGS, strategy=strategy_name, model_family='lightgbm')
        validation_snapshot = build_validation_snapshot(val_data, target)
        log_results_to_mlflow(
            results,
            run_name=f"LightGBM | {strategy_name}",
            params=params_to_log,
            tags=tags,
            extra_metrics=extra_metrics or None,
            feature_frame=X_val,
            validation_frame=validation_snapshot,
            dataset_major_only=major_only_flag,
            dataset_english_only=english_only_flag,
        )

    return results  # ✅ correctly indented inside the function

print("✅ LightGBM (Randomized) framework ready")



In [ ]:
# =========================
# Run the LightGBM strategies
# =========================
print("🎯 Training LightGBM Models (RandomizedSearchCV)")

lgb_strategy_results = []

# Strategy 1: All eras including pandemic
lgb_results_all = train_lightgbm_strategies(train_data_all, "All Eras (2015-2023)")
lgb_strategy_results.append(lgb_results_all)

# Strategy 2: Exclude pandemic era
lgb_results_no_pandemic = train_lightgbm_strategies(train_data_no_pandemic, "No Pandemic Era")
lgb_strategy_results.append(lgb_results_no_pandemic)

# Strategy 3: Weighted approach (reduced pandemic influence)
lgb_results_weighted = train_lightgbm_strategies(
    train_data_weighted,
    "Weighted (30% pandemic era)",
    sample_weights
)
lgb_strategy_results.append(lgb_results_weighted)

# Compare strategies
print("\n📊 LightGBM Strategy Comparison")
print("=" * 70)
lgb_strategy_comparison = pd.DataFrame([
    {
        'Strategy': r['strategy'],
        'RMSE': f"${r['rmse']:,.0f}",
        'MAE': f"${r['mae']:,.0f}",
        'MAPE': f"{r['mape']:.1f}%",
        'R²': f"{r['r2']:.3f}"
    }
    for r in lgb_strategy_results
])
print(lgb_strategy_comparison.to_string(index=False))

# Pick best
best_lgb_strategy_idx = min(
    range(len(lgb_strategy_results)),
    key=lambda i: lgb_strategy_results[i]['rmse']
)
best_lgb_strategy = lgb_strategy_results[best_lgb_strategy_idx]

print(f"\n🏆 Best LightGBM Strategy: {best_lgb_strategy['strategy']}")
print(f"   Validation RMSE: ${best_lgb_strategy['rmse']:,.0f}")

# Store best model
best_lgb_model = best_lgb_strategy['model_object']

lgb_strategy_results = []


In [ ]:
# Save best models from each approach for comparison in notebook 5
import pickle
import os

print("💾 Saving Best Models for Final Comparison")
print("=" * 60)

# Create models directory if it doesn't exist
os.makedirs('../models', exist_ok=True)

# Collect all available results
all_available_results = []

# Find best Ridge model
best_ridge_idx = min(range(len(strategy_results)), key=lambda i: strategy_results[i]['rmse'])
best_ridge = strategy_results[best_ridge_idx]
all_available_results.append(('ridge', best_ridge))

# Find best Decision Tree model  
best_dt_idx = min(range(len(dt_strategy_results)), key=lambda i: dt_strategy_results[i]['rmse'])
best_dt = dt_strategy_results[best_dt_idx]
all_available_results.append(('decision_tree', best_dt))

# Find best Random Forest model
best_rf_idx = min(range(len(rf_strategy_results)), key=lambda i: rf_strategy_results[i]['rmse'])
best_rf = rf_strategy_results[best_rf_idx]
all_available_results.append(('random_forest', best_rf))

# Find best XGBoost model
if len(xgb_strategy_results) > 0:
    best_xgb_idx = min(range(len(xgb_strategy_results)), key=lambda i: xgb_strategy_results[i]['rmse'])
    best_xgb = xgb_strategy_results[best_xgb_idx]
    all_available_results.append(('xgboost', best_xgb))

# Find best LightGBM model
if len(lgb_strategy_results) > 0:
    best_lgb_idx = min(range(len(lgb_strategy_results)), key=lambda i: lgb_strategy_results[i]['rmse'])
    best_lgb = lgb_strategy_results[best_lgb_idx]
    all_available_results.append(('lightgbm', best_lgb))

# Save best models and their metadata
models_to_save = {}
for model_name, best_result in all_available_results:
    models_to_save[model_name] = {
        'model': best_result['model_object'],
        'strategy': best_result['strategy'],
        'rmse': best_result['rmse'],
        'mae': best_result['mae'],
        'mape': best_result['mape'],
        'r2': best_result['r2'],
        'type': best_result.get('model_type', model_name.replace('_', ' ').title())
    }

# Save each model
for model_name, model_data in models_to_save.items():
    with open(f'../models/best_{model_name}_model.pkl', 'wb') as f:
        pickle.dump(model_data, f)
    
    print(f"✅ Saved {model_data['type']} ({model_data['strategy']})")
    print(f"   RMSE: ${model_data['rmse']:,.0f}, R²: {model_data['r2']:.3f}")

# Save feature columns for consistency
with open('../models/feature_columns.pkl', 'wb') as f:
    pickle.dump(feature_cols, f)

print(f"\n📁 Models saved to ../models/ directory")
saved_models = [f"best_{name}_model.pkl" for name in models_to_save.keys()]
for model_file in saved_models:
    print(f"   • {model_file}")
print(f"   • feature_columns.pkl")

print(f"\n🔄 Use these models in notebook 5 for final comparison")
print(f"📊 Total models available: {len(models_to_save)}")

# Show quick comparison of all models
if len(models_to_save) > 1:
    print(f"\n🏆 Quick Model Comparison (Best RMSE):")
    comparison_list = [(name, data['rmse'], data['r2']) for name, data in models_to_save.items()]
    comparison_list.sort(key=lambda x: x[1])  # Sort by RMSE
    
    for i, (name, rmse, r2) in enumerate(comparison_list, 1):
        print(f"   {i}. {name.replace('_', ' ').title()}: RMSE ${rmse:,.0f}, R² {r2:.3f}")
    
    best_overall = comparison_list[0]
    print(f"\n🥇 Overall Best: {best_overall[0].replace('_', ' ').title()}")
    print(f"   RMSE: ${best_overall[1]:,.0f}")
    print(f"   R²: {best_overall[2]:.3f}")